In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.stats import kurtosis

# Configuration
FS = 2000       # Sampling Rate (Hz)
DURATION = 2.0  # Duration (seconds)
N = int(FS * DURATION)          
t = np.linspace(0, DURATION, N, endpoint=False) 

print("Cell 1 Complete: Libraries loaded.")

Cell 1 Complete: Libraries loaded.


In [3]:
def generate_signal(has_fault=False):
    # Base Signal
    signal = 0.5 * np.sin(2 * np.pi * 50 * t)
    # Noise
    noise = np.random.normal(0, 0.3, N)
    signal += noise
    # Fault
    if has_fault:
        fault_signature = 0.4 * np.sin(2 * np.pi * 120 * t)
        signal += fault_signature
    return signal

# This defines the variables your later code needs!
normal_signal = generate_signal(has_fault=False)
faulty_signal = generate_signal(has_fault=True)

print("Cell 2 Complete: 'normal_signal' and 'faulty_signal' are now defined.")

Cell 2 Complete: 'normal_signal' and 'faulty_signal' are now defined.


In [6]:
def compute_fft(signal):
    # Compute amplitude spectrum using NumPy's FFT
    Y = np.fft.rfft(signal)
    amps = np.abs(Y)
    freqs = np.fft.rfftfreq(N, d=1.0/FS)
    return freqs, amps

print("Cell 2A Complete: 'compute_fft' helper defined.")

Cell 2A Complete: 'compute_fft' helper defined.


In [7]:
from scipy.stats import kurtosis

def extract_features(signal, fs):
    features = {}
    
    # --- TIME DOMAIN FEATURES ---
    # 1. RMS (Root Mean Square) - The "Loudness" of the vibration
    features['rms'] = np.sqrt(np.mean(signal**2))
    
    # 2. Kurtosis - The "Spikiness" (High kurtosis = sharp impacts like clicks)
    features['kurtosis'] = kurtosis(signal)
    
    # 3. Crest Factor - Peak vs. Average (Impact intensity)
    peak = np.max(np.abs(signal))
    features['crest_factor'] = peak / features['rms']
    
    # --- FREQUENCY DOMAIN FEATURES ---
    # 4. Dominant Frequency - Where is the energy concentrated?
    # We reuse our FFT logic here
    freqs, amps = compute_fft(signal)
    
    # Find the index of the tallest spike
    dominant_index = np.argmax(amps)
    features['peak_freq'] = freqs[dominant_index]
    
    return features

# Test it on our two signals
normal_feats = extract_features(normal_signal, FS)
faulty_feats = extract_features(faulty_signal, FS)

print("--- DIAGNOSTIC REPORT ---")
print(f"NORMAL Signal:  RMS={normal_feats['rms']:.3f}, Kurtosis={normal_feats['kurtosis']:.3f}, Peak Freq={normal_feats['peak_freq']} Hz")
print(f"FAULTY Signal:  RMS={faulty_feats['rms']:.3f}, Kurtosis={faulty_feats['kurtosis']:.3f}, Peak Freq={faulty_feats['peak_freq']} Hz")

--- DIAGNOSTIC REPORT ---
NORMAL Signal:  RMS=0.469, Kurtosis=-0.450, Peak Freq=50.0 Hz
FAULTY Signal:  RMS=0.538, Kurtosis=-0.391, Peak Freq=50.0 Hz
